In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

## 종업원수 데이터 전처리 & 입지계수 및 다양성지수 생성

In [ ]:
employee_df = pd.read_csv('/content/drive/MyDrive/데멘토/종업원수.csv', encoding='cp949')

In [ ]:
employee_df.columns = ['기준년월', '시도명', '시군구명', '행정동명', '업종대분류코드', '업종중분류코드', '업종대분류', '업종중분류', '가공기업구분코드', '가공기업구분', '업력구간코드',
                   '업력구간', '총기업수', '종업원수', '종업원평균수', '입사수', '입사평균수', '퇴직수', '퇴직평균수', '평균임금', '등록일자', '작업자명']

employee_df = employee_df.drop(['업종대분류코드', '업종중분류코드', '가공기업구분코드', '업력구간코드', '업력구간', '작업자명', '등록일자'], 
                               axis = 'columns')

employee_grouped = employee_df.groupby(['기준년월', '시도명', '시군구명', '행정동명', '업종대분류', '업종중분류', '가공기업구분'], 
                                       as_index = False).agg({'총기업수': np.sum, '종업원수': np.sum, '입사수': np.sum, '퇴직수': np.sum})

wage = employee_df.groupby(['기준년월', '시도명', '시군구명', '행정동명', '업종대분류', '업종중분류', '가공기업구분'], as_index=False).apply(lambda x: (x.종업원수 * x.평균임금).sum()).iloc[:, -1]                                       
employee_grouped['총임금'] = wage
employee_grouped['평균임금'] = employee_grouped.총임금 / employee_grouped.종업원수
employee_grouped

,기준년월,시도명,시군구명,행정동명,업종대분류,업종중분류,가공기업구분,총기업수,종업원수,입사수,퇴직수,총임금,평균임금
0,2015-11,강원,강릉시,강남동,건설업,전문직별 공사업,소기업,23,152,4,8,2.522793e+09,1.659732e+07
1,2015-11,강원,강릉시,강남동,건설업,전문직별 공사업,중기업,1,11,0,0,2.050747e+08,1.864315e+07
2,2015-11,강원,강릉시,강남동,건설업,종합 건설업,소기업,7,31,0,1,5.638020e+08,1.818716e+07
3,2015-11,강원,강릉시,강남동,금융 및 보험업,금융업,소기업,1,11,0,0,4.189493e+08,3.808630e+07
4,2015-11,강원,강릉시,강남동,농업; 임업 및 어업,임업,소기업,1,5,1,1,3.856534e+07,7.713067e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275994,2022-05,충북,충주시,호암.직동,정보통신업,출판업,소기업,1,3,0,0,1.521600e+08,5.072000e+07
7275995,2022-05,충북,충주시,호암.직동,제조업,식료품 제조업,소기업,2,38,1,2,1.059037e+09,2.786940e+07
7275996,2022-05,충북,충주시,호암.직동,제조업,자동차 및 트레일러 제조업,소기업,1,17,0,0,3.761760e+08,2.212800e+07
7275997,2022-05,충북,충주시,호암.직동,협회 및 단체; 수리 및 기타 개인 서비스업,기타 개인 서비스업,중기업,1,10,0,1,2.607920e+08,2.607920e+07


In [ ]:
## 파일 저장
employee_grouped.to_csv('종업원수_prep.csv', index=False)

In [ ]:
## 입지계수, 다양성지수 생성
employee_grouped = employee_df.groupby(['기준년월', '시도명', '시군구명', '업종대분류'], 
                                       as_index = False).agg({'총기업수': np.sum, '종업원수': np.sum})

## 입지계수 칼럼 생성                                       
전국_employee = employee_grouped.groupby(['기준년월'])['종업원수'].sum().reset_index()
전국_산업_employee = employee_grouped.groupby(['기준년월','업종대분류'])['종업원수'].sum().reset_index()
전국_지역_employee = employee_grouped.groupby(['기준년월','시도명','시군구명'])['종업원수'].sum().reset_index()

employee_지표 = pd.merge(employee_grouped,전국_employee,on = ["기준년월"])
employee_지표 = pd.merge(employee_지표,전국_산업_employee,on = ["기준년월", "업종대분류"])
employee_지표 = pd.merge(employee_지표,전국_지역_employee,on = ['기준년월', '시도명', '시군구명'])
employee_지표.columns = ['기준년월', '시도명', '시군구명', '업종대분류', '총기업수', '전국_산업_지역_종업원수', '전국_종업원수', '전국_산업_종업원수', '전국_지역_종업원수']
employee_지표['입지계수'] = (employee_지표['전국_산업_지역_종업원수'] / employee_지표['전국_지역_종업원수']) / (employee_지표['전국_산업_종업원수'] / employee_지표['전국_종업원수'])
employee_지표.drop(columns=['전국_산업_지역_종업원수', '전국_종업원수', '전국_산업_종업원수', '전국_지역_종업원수'], inplace=True)
employee_지표

## 다양성지수 컬럼 생성
전국_산업_지역_employee = employee_grouped.groupby(['기준년월', '시도명', '시군구명', '업종대분류',])['종업원수'].sum().reset_index()
employee_다양성지수 = pd.merge(전국_지역_employee, 전국_산업_지역_employee, on=['기준년월', '시도명', '시군구명'])
employee_다양성지수['다양성지수'] = (employee_다양성지수['종업원수_y'] / employee_다양성지수['종업원수_x']) ** 2
employee_다양성지수.drop(columns=['종업원수_x', '종업원수_y'], inplace=True)
employee_다양성지수 = employee_다양성지수.groupby(['기준년월', '시도명', '시군구명'])['다양성지수'].sum().reset_index()
employee_다양성지수['다양성지수'] = 1 / employee_다양성지수['다양성지수']
## 다양성지수 전국 평균 계산
전국평균다양성지수 = employee_다양성지수.groupby(['기준년월'])['다양성지수'].mean().reset_index()
employee_다양성지수 = pd.merge(employee_다양성지수,전국평균다양성지수, on=['기준년월'])
employee_다양성지수.columns=['기준년월', '시도명', '시군구명', '다양성지수', '다양성지수_전국평균']

## 데이터프레임 병합
employee_지표 = pd.merge(employee_지표, employee_다양성지수,on=['기준년월', '시도명', '시군구명'])
employee_지표

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'종업원수_x'} in the result is deprecated and will raise a MergeError in a future version.
  if sys.path[0] == '':


,기준년월,시도명,시군구명,업종대분류,총기업수,입지계수,다양성지수,다양성지수_전국평균
0,2015-11,강원,강릉시,건설업,396,2.604523,7.132414,4.510234
1,2015-11,강원,강릉시,광업,13,6.265855,7.132414,4.510234
2,2015-11,강원,강릉시,교육 서비스업,11,1.321261,7.132414,4.510234
3,2015-11,강원,강릉시,금융 및 보험업,17,0.324925,7.132414,4.510234
4,2015-11,강원,강릉시,농업; 임업 및 어업,24,3.819466,7.132414,4.510234
...,...,...,...,...,...,...,...,...
345112,2022-05,충북,충주시,전기; 가스; 증기 및 공기조절 공급업,3,0.280479,2.789349,5.207340
345113,2022-05,충북,충주시,전문; 과학 및 기술 서비스업,61,0.432539,2.789349,5.207340
345114,2022-05,충북,충주시,정보통신업,18,0.127233,2.789349,5.207340
345115,2022-05,충북,충주시,제조업,338,1.646578,2.789349,5.207340


In [ ]:
## 파일 저장
employee_지표.to_csv('고용위기지표.csv', index=False)

## 기업현황 데이터 

In [ ]:
## 파일 읽기
기업현황_df = pd.read_csv('/content/drive/MyDrive/데멘토/기업현황.csv', encoding='cp949')

new_df = 기업현황_df[['STDR_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'INDUTY_LCLAS_NM', 'INDUTY_MLSFC_NM', 'PDSMLPZ_SCTN_NM', 'PRCSS_LTRS_CO', 'PRCSS_MIDPZ_CO', 'PRCSS_MSLENTR_CO', 'PRCSS_SSLENTR_CO']]
new_df.columns = ['Date', '시도명', '시군구명', '업종대분류명', '업종중분류명', '업력구간명', '대기업수', '중견기업수', '중기업수', '소기업수']

## 2016년 이후 데이터 추출
new_df = new_df[new_df.Date.apply(lambda x : int(x[:4]) > 2015 )]
new_df = new_df.groupby(['Date', '시도명', '시군구명', '업종대분류명', '업종중분류명', '업력구간명']).sum().reset_index()
new_df

,Date,시도명,시군구명,업종대분류명,업종중분류명,업력구간명,대기업수,중견기업수,중기업수,소기업수
0,2016-01,강원,강릉시,건설업,전문직별 공사업,10년 이상 20년 미만,0,0,4,154
1,2016-01,강원,강릉시,건설업,전문직별 공사업,1년 미만,0,0,0,21
2,2016-01,강원,강릉시,건설업,전문직별 공사업,1년 이상 2년 미만,0,0,0,13
3,2016-01,강원,강릉시,건설업,전문직별 공사업,20년 이상 30년 미만,0,0,3,28
4,2016-01,강원,강릉시,건설업,전문직별 공사업,2년 이상 5년 미만,0,0,0,45
...,...,...,...,...,...,...,...,...,...,...
6008190,2022-07,충북,충주시,협회 및 단체; 수리 및 기타 개인 서비스업,협회 및 단체,30년 이상 40년 미만,0,0,1,2
6008191,2022-07,충북,충주시,협회 및 단체; 수리 및 기타 개인 서비스업,협회 및 단체,40년 이상 50년 미만,0,0,1,0
6008192,2022-07,충북,충주시,협회 및 단체; 수리 및 기타 개인 서비스업,협회 및 단체,50년 이상 60년 미만,0,0,0,1
6008193,2022-07,충북,충주시,협회 및 단체; 수리 및 기타 개인 서비스업,협회 및 단체,5년 이상 10년 미만,0,0,2,3


In [ ]:
## 파일저장
new_df.to_csv('기업현황_최종.csv', index = False)

## 전체밀집지역기업리스트 전처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데멘토/전체밀집지역기업리스트.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
## 업종 대분류만 남기기
df.업종명 = df.업종명.fillna("")
df.업종명 = df.업종명.apply(lambda x : x.split('/')[0]) 

## 기업현황에 있는 업종명이랑 같게 전처리 (태블로에서 join 용)
df.업종명 = df.업종명.replace('도매및소매업', '도매 및 소매업')
df.업종명 = df.업종명.replace('전문,과학및기술서비스업', '전문; 과학 및 기술 서비스업')
df.업종명 = df.업종명.replace('사업시설관리,사업지원및임대서비스업', '사업시설 관리; 사업 지원 및 임대 서비스업')
df.업종명 = df.업종명.replace('교육서비스업', '교육 서비스업')
df.업종명 = df.업종명.replace('예술,스포츠및여가관련서비스업', '예술; 스포츠 및 여가관련 서비스업')
df.업종명 = df.업종명.replace('숙박및음식점업', '숙박 및 음식점업')
df.업종명 = df.업종명.replace('운수및창고업', '운수 및 창고업')
df.업종명 = df.업종명.replace('전기,가스,증기및공기조절공급업', '전기; 가스; 증기 및 공기조절 공급업')
df.업종명 = df.업종명.replace('협회및단체,수리및기타개인서비스업', '협회 및 단체; 수리 및 기타 개인 서비스업')
df.업종명 = df.업종명.replace('정보통신', '운수 및 정보통신업')
df.업종명 = df.업종명.replace('수도,하수및폐기물처리,원료재생업', '수도; 하수 및 폐기물 처리; 원료 재생업')
df.업종명 = df.업종명.replace('농업,임업및어업', '농업; 임업 및 어업')
df.업종명 = df.업종명.replace('보건업및사회복지서비스업', '보건업 및 사회복지 서비스업')

## 재무관련 데이터 추출 
매출액증가율_df = df[[ '시도명', '시군구명', '업종명',
                '매출액증가율_2016',
                '매출액증가율_2017',
                '매출액증가율_2018',
                '매출액증가율_2019',
                '매출액증가율_2020',
                '매출액증가율_2021',
                '매출액증가율_2022']]
급여액_1인평균_df = df[['시도명', '시군구명', '업종명',
                '1인평균급여액_2016',
                '1인평균급여액_2017',
                '1인평균급여액_2018',
                '1인평균급여액_2019',
                '1인평균급여액_2020',
                '1인평균급여액_2021',
                '1인평균급여액2022']]
부채비율_df = df[['시도명', '시군구명', '업종명',
                '부채비율_2016',
                '부채비율_2017',
                '부채비율_2018',
                '부채비율_2019',
                '부채비율_2020',
                '부채비율_2021',
                '부채비율_2022']]
매출액_df = df[['시도명', '시군구명', '업종명',
                '매출액_2016',
                '매출액_2017',
                '매출액_2018',
                '매출액_2019',
                '매출액_2020',
                '매출액_2021',
                '매출액_2022']]

## 각각의 피쳐들을 하나의 컬럼으로 합침
매출액증가율_df = 매출액증가율_df.melt(id_vars=['시도명', '시군구명', '업종명'], value_vars=[
    '매출액증가율_2016',
    '매출액증가율_2017',
    '매출액증가율_2018',
    '매출액증가율_2019',
    '매출액증가율_2020',
    '매출액증가율_2021',
    '매출액증가율_2022'])
급여액_1인평균_df = 급여액_1인평균_df.melt(id_vars=['시도명', '시군구명', '업종명'], value_vars=[
    '1인평균급여액_2016',
    '1인평균급여액_2017',
    '1인평균급여액_2018',
    '1인평균급여액_2019',
    '1인평균급여액_2020',
    '1인평균급여액_2021',
    '1인평균급여액2022'])
부채비율_df = 부채비율_df.melt(id_vars=['시도명', '시군구명', '업종명'], value_vars=[
    '부채비율_2016',
    '부채비율_2017',
    '부채비율_2018',
    '부채비율_2019',
    '부채비율_2020',
    '부채비율_2021',
    '부채비율_2022'])
매출액_df = 매출액_df.melt(id_vars=['시도명', '시군구명', '업종명'], value_vars=[
    '매출액_2016',
    '매출액_2017',
    '매출액_2018',
    '매출액_2019',
    '매출액_2020',
    '매출액_2021',
    '매출액_2022'])


## 년도 칼럼 생성
매출액증가율_df['year'] = 매출액증가율_df['variable'].apply(lambda x : x[-4:])
매출액증가율_df.rename(columns={'value':'매출액증가율'}, inplace=True)
매출액증가율_df = 매출액증가율_df.drop(['variable'], axis = 1)

급여액_1인평균_df['year'] = 급여액_1인평균_df['variable'].apply(lambda x : x[-4:])
급여액_1인평균_df.rename(columns={'value':'1인평균급여액'}, inplace=True)
급여액_1인평균_df = 급여액_1인평균_df.drop(['variable'], axis = 1)

부채비율_df['year'] = 부채비율_df['variable'].apply(lambda x : x[-4:])
부채비율_df.rename(columns={'value':'부채비율'}, inplace=True)
부채비율_df = 부채비율_df.drop(['variable'], axis = 1)

매출액_df['year'] = 매출액_df['variable'].apply(lambda x : x[-4:])
매출액_df.rename(columns={'value':'매출액'}, inplace=True)
매출액_df = 매출액_df.drop(['variable'], axis = 1)

## 집계
매출액증가율_df = 매출액증가율_df.groupby(['year', '시도명', '시군구명', '업종명'])['매출액증가율'].mean().reset_index()
급여액_1인평균_df = 급여액_1인평균_df.groupby(['year', '시도명', '시군구명', '업종명'])['1인평균급여액'].mean().reset_index()
부채비율_df = 부채비율_df.groupby(['year', '시도명', '시군구명', '업종명'])['부채비율'].mean().reset_index()
매출액_df = 매출액_df.groupby(['year', '시도명', '시군구명', '업종명'])['매출액'].sum().reset_index()

## 집계된 데이터프레임 하나로 합침
all_df = 매출액증가율_df[['year', '시도명', '시군구명', '업종명']]
all_df['매출액증가율'] = 매출액증가율_df['매출액증가율']
all_df['1인평균급여액'] = 급여액_1인평균_df['1인평균급여액']
all_df['부채비율'] = 부채비율_df['부채비율']
all_df['매출액'] = 매출액_df['매출액']
all_df

,year,시도명,시군구명,업종명,매출액증가율,1인평균급여액,부채비율,매출액
0,2016,강원도,강릉시,,33.242236,2.150394e+06,-360.891950,116608453.0
1,2016,강원도,강릉시,건설업,88.680976,2.023466e+06,55.594841,77569486.0
2,2016,강원도,강릉시,농업; 임업 및 어업,NaN,NaN,NaN,0.0
3,2016,강원도,강릉시,도매 및 소매업,23.965568,1.899104e+06,70.118216,57824986.0
4,2016,강원도,강릉시,부동산업,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
19672,2022,충청북도,충주시,운수 및 창고업,NaN,2.678142e+06,NaN,0.0
19673,2022,충청북도,충주시,전문; 과학 및 기술 서비스업,NaN,2.499210e+06,NaN,0.0
19674,2022,충청북도,충주시,정보통신업,NaN,3.002906e+06,NaN,0.0
19675,2022,충청북도,충주시,제조업,NaN,3.215806e+06,NaN,0.0


In [ ]:
## 파일 저장
all_df.to_csv('밀집지역기업리스트_재무.csv', index=False)